In [1]:
%pip install lobsang openai python-dotenv

Note: you may need to restart the kernel to use updated packages.


# Basics

In this notebook, we'll cover the basics of lobsang. As a LLM we use OpenAI's "gpt-3.5-turbo" model, so you'll need an [OpenAI API key](https://platform.openai.com/account/api-keys) to run this notebook (set it in the cell below).
This is not a free service, but all examples in this notebook should cost less than $0.01 to run. Nevertheless, you should set a [usage limit](https://platform.openai.com/account/billing/limits) to avoid any surprises.

In [2]:
import os

from dotenv import load_dotenv
from openai import ChatCompletion

from lobsang import Chat, LLM
from lobsang.messages import SystemMessage, UserMessage, AssistantMessage

load_dotenv()

# Load OpenAI API key from .env file (please update .env file with your own API key)
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
assert OPENAI_API_KEY, "Please set OPENAI_API_KEY in .env file"

f"All set! 🎉 Let's get started! 🚀 OPENAI_API_KEY={OPENAI_API_KEY[:15]}..."

"All set! 🎉 Let's get started! 🚀 OPENAI_API_KEY=sk-PbRY4M6AH6Xh..."

## Message Class

Before we get started, let's take a look at the `Message` class. This is the abstract base class for all messages in lobsang. It has three subclasses: `SystemMessage`, `UserMessage`, and `AssistantMessage`.

1. `SystemMessage` is a special case and is used to set the model's behavior (There should only be one `SystemMessage` per `Chat` instance at the beginning of the conversation.)
1. `UserMessage` is used for messages that are created by the user/developer.
1. `AssistantMessage` is used for messages that are sent generated by the llm.

In [3]:
system_message = SystemMessage("You are a helpful assistant.")
user_message = UserMessage("Hello, my name is Bark Twain.")
assistant_message = AssistantMessage("Hello Bark, I'm Lobsang.")

messages = [system_message, user_message, assistant_message]

# The message class implements the __repr__ contract/interface that returns a string representation of the message (useful for debugging)
print("__repr__")
print(*map(repr, messages), sep="\n") # 💡 '*...' unpacks the list into individual arguments (i.e. print(*[1,2,3]) is the same as print(1,2,3)) 
    
# And the __str__ contract/interface as well that returns a chat-formatted string representation of the message (<ROLE>: <TEXT>)
print("\n__str__")
print(*messages, sep="\n") # 💡 print automatically calls __str__ on objects

__repr__
SystemMessage(role=system, text=You are a helpful assistant., info={})
UserMessage(role=user, text=Hello, my name is Bark Twain., info={})
AssistantMessage(role=assistant, text=Hello Bark, I'm Lobsang., info={})

__str__
SYSTEM: You are a helpful assistant.
USER: Hello, my name is Bark Twain.
ASSISTANT: Hello Bark, I'm Lobsang.


☝️ As you can see, each message has a `role`, `text` and `info` attribute. 
1. `role` is the role of the message sender (i.e. "user", "assistant", or "system")
2. `text` is the text of the message
3. `info` is a dictionary that can be used to store additional information about the message

## LLM Wrapper

For experiments and testing, we also provide a FakeLLM which can be imported via `from lobsang.llms import FakeLLM`. However, in this notebook we'll use OpenAI's API. So before we can start chatting, we need to wrap the openai api to make it compatible with lobsang. This can be done with lobsang's `LLM` class as follows:

In [4]:
class OpenAI(LLM):
    """Wrapper for OpenAI API"""
    def __init__(self):
        self.api_key = OPENAI_API_KEY
        self.model = "gpt-3.5-turbo"
        
    def chat(self, messages) -> (str, dict):
        """
        Chat with OpenAI API
        
        Takes a list of messages and converts them to the format required by the OpenAI API.
        Then, calls the API and retrieves the response text to return it to the caller.
        """
        # Prepare messages
        messages = [{'role': m.role, 'content': m.text} for m in messages]
        
        # Call OpenAI API
        res = ChatCompletion.create(
            model=self.model,
            api_key=self.api_key,
            messages=messages,
        )
        
        # Return text response
        return res.choices[0].message.content, {}

**Note**: In this example, we return an empty dictionary as the second return value. However, all keys added to the dictionary will end up in the `info` attribute of the corresponding `AssistantMessage` that is created by the chat instance from the response text. This can be used to store additional information about the message. This is implementation is also provided under `lobsang.llms.openai` and can be imported via `from lobsang import OpenAI`.

## Chat Instance

Now, let's bring all the pieces together and create a `Chat` instance. In this example, we'll call chat directly with a string, i.e. `chat("Ping")`. 
This is equivalent to `chat.run([UserMessage("Ping")])[1]` or `chat.run(["Ping"])[1]` (we'll get to the `run` method in a bit), but since this is verbose we provide a shortcut for this common use case with the `__call__` method, i.e. `chat("Ping")`.

In [5]:
# We first create a chat instance with the LLM wrapper from above
chat = Chat(llm=OpenAI())  # 👈 Note that we don't pass a system message here, so the model will use its default behavior "You are a helpful assistant."

# And we are ready to go! 🚶 Let's chat! 🗣
res = chat("Ping.")

# Let's see what we got back
res

AssistantMessage(role=assistant, text=Pong!, info={'original': 'Pong!', 'directive': TextDirective})

☝️ As you can see, the response is a single `AssistantMessage` instance. It has a `role`, `text` and `info` attribute. The `role` is "assistant" as we'd suspect. The `text` is the response from the llm. And the `info` is a dictionary with additional information about the message, for example to help with debugging.

Alright, in the next example, we'll initialize the chat instance with a `SystemMessage` to set the model's behavior. 
Let's make it a fancy wizard 🧙‍♂️ that speaks in a medieval style:

In [6]:
# First, we need to write a system message that sets the model's behavior
system_message = "You are a fancy wizard and talk in a medieval style."

# Then, we create a chat instance again with the LLM wrapper from above and pass the system message
chat = Chat(llm=OpenAI(), system_message=system_message)

# And we are ready to go! 🚶 Let's chat! 🗣
res = chat("What is you favourite spell?")

print(res.text)

Ah, dear traveler, thou hath inquired about mine favorite spell! Verily, mine heart doth rejoice at the mention of it. The spell that doth capture mine fancy and ignite the flames of enchantment within my soul is the wondrous incantation known as "Aurora Borealis."

With the power of this spell, I am able to summon the mesmerizing lights of the Northern Sky. Erupting in vibrant hues of azure, emerald, and amethyst, the ethereal display dances across the heavens, filling the night with its celestial majesty.

As I intone the ancient words and weave my hands in intricate gestures, the very fabric of reality bends to my will. Pure magic enlivens the air, swirling and twirling with mystical energy, until the breathtaking splendor of the Aurora Borealis emerges from the mystical realm to grace ours.

This spell, dear interlocutor, serves not only as a magnificent spectacle to behold but also acts as a beacon of hope and wonder to all who witness it. The Aurora Borealis fills the hearts of t

☝️ Awesome! 🎉 In a couple of lines of code, we've created a fancy wizard ‍🧙‍♂️

Now, it's time to let you in on a little secret 👀 The `Chat` class actually inherits from `list` 🤯 and contains the chat history.
This means, we can do all the usual list operations on it.

> **Note:** Please note that not all list operations are supported. For example, sorting is not supported. If you are missing a list operation, please open an issue on GitHub. Thanks! 🙏

👇 Now, let's take a look at the chat history:

In [7]:
print(chat)

USER: What is you favourite spell?
ASSISTANT: Ah, dear traveler, thou hath inquired about mine favorite spell! Verily, mine heart doth rejoice at the mention of it. The spell that doth capture mine fancy and ignite the flames of enchantment within my soul is the wondrous incantation known as "Aurora Borealis."

With the power of this spell, I am able to summon the mesmerizing lights of the Northern Sky. Erupting in vibrant hues of azure, emerald, and amethyst, the ethereal display dances across the heavens, filling the night with its celestial majesty.

As I intone the ancient words and weave my hands in intricate gestures, the very fabric of reality bends to my will. Pure magic enlivens the air, swirling and twirling with mystical energy, until the breathtaking splendor of the Aurora Borealis emerges from the mystical realm to grace ours.

This spell, dear interlocutor, serves not only as a magnificent spectacle to behold but also acts as a beacon of hope and wonder to all who witness

☝️ As you can see, the chat history contains our previous message and the response from the model.

👇 The following shows a selection of supported list operations:

In [8]:
# We can iterate over the chat history
for message in chat:
    print(message)

USER: What is you favourite spell?
ASSISTANT: Ah, dear traveler, thou hath inquired about mine favorite spell! Verily, mine heart doth rejoice at the mention of it. The spell that doth capture mine fancy and ignite the flames of enchantment within my soul is the wondrous incantation known as "Aurora Borealis."

With the power of this spell, I am able to summon the mesmerizing lights of the Northern Sky. Erupting in vibrant hues of azure, emerald, and amethyst, the ethereal display dances across the heavens, filling the night with its celestial majesty.

As I intone the ancient words and weave my hands in intricate gestures, the very fabric of reality bends to my will. Pure magic enlivens the air, swirling and twirling with mystical energy, until the breathtaking splendor of the Aurora Borealis emerges from the mystical realm to grace ours.

This spell, dear interlocutor, serves not only as a magnificent spectacle to behold but also acts as a beacon of hope and wonder to all who witness

In [9]:
# We can also get the length of the chat history
len(chat)

2

In [10]:
# We can index into it (or slice it, i.e chat[i:j])
chat[0]

UserMessage(role=user, text=What is you favourite spell?, info={'original': 'What is you favourite spell?', 'directive': TextDirective})

In [11]:
# and replace messages
chat[0] = UserMessage("I'm a new message")
print(chat)

USER: I'm a new message
ASSISTANT: Ah, dear traveler, thou hath inquired about mine favorite spell! Verily, mine heart doth rejoice at the mention of it. The spell that doth capture mine fancy and ignite the flames of enchantment within my soul is the wondrous incantation known as "Aurora Borealis."

With the power of this spell, I am able to summon the mesmerizing lights of the Northern Sky. Erupting in vibrant hues of azure, emerald, and amethyst, the ethereal display dances across the heavens, filling the night with its celestial majesty.

As I intone the ancient words and weave my hands in intricate gestures, the very fabric of reality bends to my will. Pure magic enlivens the air, swirling and twirling with mystical energy, until the breathtaking splendor of the Aurora Borealis emerges from the mystical realm to grace ours.

This spell, dear interlocutor, serves not only as a magnificent spectacle to behold but also acts as a beacon of hope and wonder to all who witness it. The Au

☝️ Other operations are also supported, please try them out yourself. For example, append(), extend() and copy() are supported as well.

👇 Now that we know how to call chat and how to access the chat history, we'll take a look at one more useful feature.
Most of the time you want your message as well as the response automatically added to the chat history. So this is the default behavior of the `Chat` class. However, sometimes you might want to call the model without adding the message to the chat history. For example, if you want to explore variations of a message. For this purpose you can set the `append` flag/parameter to `False` when calling chat: `chat("Ping", append=False)`. This will return the response message without adding anything to the chat history.

In [12]:
chat = Chat(llm=OpenAI())

res = chat("What is 1+1?", append=False)

print("Response:", res.text)  
print(len(chat))  # 👈 Will be 0 as we didn't append the message

Response: 1+1 equals 2.
0


In [13]:
# Now, let's append the message and check the length again
chat("What is 1+1?")
print(chat)
print(len(chat))  # 👈 Will be 1 as we appended the message

USER: What is 1+1?
ASSISTANT: 1 + 1 equals 2.
2


### Validate() Method

Sometimes, it might be necessary to validate the chat history. For example, if you want to change the chat history manually. In this case, you can call `chat.validate()`. This will check if the chat history is valid and raise an error if it is not. Primarily, this will check if the chat history is consistent, i.e. if each message has a response and that the chat history starts with a `UserMessage` and ends with an `AssistantMessage` (otherwise the chat history would be incomplete / inconsistent). For more information and examples,  please refer to the [documentation](https://docs.convokit.cornell.edu/python/chat.html#convokit.chat.Chat.validate).

**Note:** This is a very strict validation, it is therefore not used internally but, for example, can be useful for debugging. Also, invalid chat history can still be used with the `Chat` class. However, it might lead to unexpected behavior.

Let's check out an example:

In [14]:
# New chat instance
chat = Chat(llm=OpenAI())

# Let's make the chat history invalid by extending it
invalid = [UserMessage("Ping"), AssistantMessage("Pong"), UserMessage("Ping")] # 👈 Missing response for last message
chat.extend(invalid)

# Validate the chat history
try:
    chat.validate()  # 👈 Will raise an error
except Exception as e:
    print("Error:", e)

Error: Chat is missing an assistant message at the end.


In [15]:
# Let's take a look at the current chat history (before we fix it)
print(chat)

USER: Ping
ASSISTANT: Pong
USER: Ping


In [16]:
# Now, let's fix the chat history
chat.append(AssistantMessage("Pong"))

# and re-validate it
chat.validate()  # Finger's crossed 🤞

# Let's take a look at the chat history again
print(chat)

USER: Ping
ASSISTANT: Pong
USER: Ping
ASSISTANT: Pong


### Run() Method

In addition, to passing a single message to the `Chat` class, you can also pass a list of messages. For each unanswered message (we'll explain what this means in a second), the `Chat` class will call the model and append the message as well as the response to the conversation which is returned in the end. If the `append` flag/parameter is set to `True` (default), the messages and responses will also be appended to the chat history. 

For this, we can use the `run` method which takes a list of messages and returns the conversation with the messages and responses from the model. Let's take a look:

In [17]:
chat = Chat(llm=OpenAI())

# Let's create a list of messages
# The first user message is already answered (i.e. it's an example on how the model should respond)
# The other user messages are unanswered (i.e. they have no corresponding assistant message) and will be answered by the model one-by-one
messages = [UserMessage("What is 1+1?"), AssistantMessage("2"), UserMessage("What is 2+2?"), UserMessage("What is 3+3?"), UserMessage("What is 4+4?")]

conversation = chat.run(messages)
print(*conversation, sep="\n")

USER: What is 1+1?
ASSISTANT: 2
USER: What is 2+2?
ASSISTANT: 4
USER: What is 3+3?
ASSISTANT: 6
USER: What is 4+4?
ASSISTANT: 8


☝️ As you can see the conversation is a list of messages. The first message is the first user message, followed by our predefined response from the model. The second message is the second user message, followed by the response from the model and so on. 

👇 If you do not want the example to be part of the conversation you can do the following:

In [18]:
example = [UserMessage("What is 1+1?"), AssistantMessage("2")]
chat = Chat(example, llm=OpenAI())  # 👈 Pass the example to the Chat as initial chat history

messages = messages[2:]  # 👈 Remove the example from the list of messages

conversation = chat.run(messages)  # 👈 This will also append the messages and responses to the chat history (as append=True by default)
print(*conversation, sep="\n")

USER: What is 2+2?
ASSISTANT: 4
USER: What is 3+3?
ASSISTANT: 6
USER: What is 4+4?
ASSISTANT: 8


☝️ Now, the example is not part of the conversation anymore. 

👇 Note that it is part of the chat history though:

In [19]:
print(chat)

USER: What is 1+1?
ASSISTANT: 2
USER: What is 2+2?
ASSISTANT: 4
USER: What is 3+3?
ASSISTANT: 6
USER: What is 4+4?
ASSISTANT: 8


## Conclusion

🤓 This concludes the first part of the tutorial. You should now be able to use the `Chat` class to interact with the model. 
In the next part, we'll discuss the concept of directives, which will send you through the roof 🚀 (Metaphorically speaking of course 😅).

If you have any question to hesitate to reach out to us on [Discord](https://discord.gg/wMHVAaqh).
If you've found a bug, a spelling mistake or suggestions on what could be improved, please open an issue or a pull request on [GitHub](https://github.com/cereisen/lobsang).

See you in the next part! 👋 We hope we got you hooked 🎣 